In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [17]:
url1 = 'http://www.op.gg/summoner/champions/userName=ktmata'
resp = requests.get(url1)
bs = BeautifulSoup(resp.text, 'html.parser')
print(bs)

 <!DOCTYPE html>

<!--
	  ,ad8888ba,   88888888ba         ,ad8888ba,    ,ad8888ba,
	 d8"'    `"8b  88      "8b       d8"'    `"8b  d8"'    `"8b
	d8'        `8b 88      ,8P      d8'           d8'
	88          88 88aaaaaa8P'      88            88
	88          88 88""""""'        88      88888 88      88888
	Y8,        ,8P 88               Y8,        88 Y8,        88
	 Y8a.    .a8P  88                Y8a.    .a88  Y8a.    .a88
	  `"Y8888Y"'   88     dev@op.gg   `"Y88888P"    `"Y88888P"

		 Copyright © 2013-2017 OP.GG. All rights reserved.
-->
<html lang="en_US">
<head>
<meta content="ktmata - Summoner's Champion Information - League of Legends" name="title"/>
<meta content="ktmata - Summoner's Champion Information - League of Legends" property="og:title"/>
<meta content="Real-time LoL Stats! Check your Summoner, MMR, Live Spectate and using powerful global League of Legends Statistics!" name="description"/>
<meta content="Real-time LoL Stats! Check your Summoner, MMR, Live Spectate and us

# Team List and User

In [18]:
Team_url = 'http://www.op.gg/spectate/list/'
Team_List = ["KT Rolster", "Samsung Galaxy","SKT T1", "Afreeca Freecs", "KONGDOO MONSTER"
             , "Longzhu", "ROX Tigers", "CJ Entus", "Jin Air GreenWings", "bbq OLIVERS"
            , "MVP", "Ever8 Winners"]

In [19]:
# Team List를 주면 거기에 소속된 선수들의 아이디 정보를 줌
def opgg_team(url, Team_List):
#     url = 'http://www.op.gg/spectate/list/'
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text, 'html.parser')
    u_id = [] # hide on bush
    u_name = [] # faker
    u_team = [] # skt?
    line = str(bs)
    for t in Team_List:
        for i in range(100):
            line = line[line.find("<div class=\"SummonerTeam\">{0}</div>".format(t)):].replace("<div class=\"SummonerTeam\">{0}</div>".format(t),"",1)
            line1 = line[line.find("data-summoner-extra="):line.find(" data-summoner-name=")].replace("data-summoner-extra=","").replace('"',"")
            line2 = line[line.find("data-summoner-name="):line.find(" data-team-name=")].replace("data-summoner-name=","").replace('"',"")
            line3 = line[line.find("data-team-name="):line.find("\">")].replace("data-team-name=","").replace('"',"")
            if (line1 == "") or (line2 == "") or (line3 == ""):
                break;
            u_name.append(line1)
            u_id.append(line2)
            u_team.append(line3)

    user_data = pd.DataFrame()
    user_data["User_team"] = u_team
    user_data["User_name"] = u_name
    user_data["User_id"] = u_id
    user_data
    return user_data

In [20]:
user_data = opgg_team(Team_url, Team_List)

In [21]:
User_team = [i for i in user_data["User_team"]]
User_name = [i for i in user_data["User_name"]]
User_id = [i for i in user_data["User_id"]]

# 2. Data

### Season

In [ ]:
## 시즌 번호
season = []
season_now = bs.findAll("li",{"class": "Item tabHeader active"})
# print(season_now)
season_before = bs.findAll("li",{"class": "Item tabHeader "})
for line in season_now+season_before:
    line = str(line)
    line = line[line.find("data-tab-show-class"):line.find(">")].replace("data-tab-show-class=","")
    season.append(line)
# print(season)

In [ ]:
Champion_data = pd.DataFrame()

In [ ]:
def opgg_champ(team, user_name, user_id):
    url = 'http://www.op.gg/summoner/champions/userName={0}'.format(user_id)
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text, 'html.parser')
    
    print(url)
    ### Champion Name (string)
    ## 챔피언 이름
    name = bs.findAll("td", {"class": "ChampionName"})
    champion_name = []
    for line in name:
        line = str(line)
        line = line[line.find("data-value="):line.find(">")].replace("data-value=",'').replace("\"","")
        champion_name.append(line)


    ### Win, Lose (float)

    wratio = bs.findAll("td", {"class":"RatioGraph Cell"})

    win = []
    lose = []
    for line in wratio:
        line = str(line)
        line1 = line[line.find("\"Text Left\">"):line.find("W<")].replace("\"Text Left\">",'')
        line2 = line[line.find("\"Text Right\">"):line.find("L<")].replace("\"Text Right\">",'')
        if line1 == "":
            line1 = "0"
        if line2 == "":
            line2 = "0"
        win.append(line1)
        lose.append(line2)
    win = [float(i) for i in win]
    lose = [float(i) for i in lose]

    # print(win)
    # print(len(win))
    # print(lose)

    ### KDA

    kda_data = bs.findAll("div", {"class":"KDA"})
    # print(kda_data)
    kill = []
    death = []
    assist =[]
    for line in kda_data:
        line = str(line)
        line1 = line[line.find("\"Kill\">"):].replace("\"Kill\">","")
        line1 = line1[:line1.find("</span")]
        line2 = line[line.find("\"Death\">"):].replace("\"Death\">","")
        line2 = line2[:line2.find("</span")]
        line3 = line[line.find("\"Assist\">"):].replace("\"Assist\">","")
        line3 = line3[:line3.find("</span")]
        kill.append(line1)
        death.append(line2)
        assist.append(line3)
    kill = [float(i) for i in kill]
    death = [float(i) for i in death]
    assist = [float(i) for i in assist]
    # print(kill)
    # print(death)
    # print(assist)

    ### other data

    data = bs.findAll("td", {"class": "Value Cell"})

    gold = []
    cs = []
    tower = []
    max_kill = []
    max_death = []
    avg_attack_damage = []
    avg_defend_damage = []
    double_kill = []
    triple_kill = []
    quadra_kill = []

    index = 0;
    for line in data:
        line = str(line)
        line = line[line.find("\"Value Cell\">"):].replace("\"Value Cell\">","")
        line = line[:line.find("</td>")].replace("\n","").replace("\t","").replace("\"","").replace(",","")
        ## 0값 넣기
        if line == "":
            line = "0"
        ## 역할군 나누기    
        if (index % 10) == 0:
            gold.append(line)
        elif (index % 10) == 1:
            cs.append(line)
        elif (index % 10) == 2:
            tower.append(line)
        elif (index % 10) == 3:
            max_kill.append(line)
        elif (index % 10) == 4:
            max_death.append(line)
        elif (index % 10) == 5:
            avg_attack_damage.append(line)
        elif (index % 10) == 6:
            avg_defend_damage.append(line)
        elif (index % 10) == 7:
            double_kill.append(line)
        elif (index % 10) == 8:
            triple_kill.append(line)
        elif (index % 10) == 9:
            quadra_kill.append(line)
        #index 증가
        index = index+1
    # float 변환
    gold = [float(i) for i in gold]
    cs = [float(i) for i in cs]
    tower = [float(i) for i in tower]
    max_kill = [float(i) for i in max_kill]
    max_death = [float(i) for i in max_death]
    avg_attack_damage = [float(i) for i in avg_attack_damage]
    avg_defend_damage = [float(i) for i in avg_defend_damage]
    double_kill = [float(i) for i in double_kill]
    triple_kill = [float(i) for i in triple_kill]
    quadra_kill = [float(i) for i in quadra_kill]

    
    #     print(line)
    #     print("********************")
    # print(len(gold))

    # 3. Make DataFrame

    Champion_data = pd.DataFrame()
    
    Champion_data["Champion_Name"] = champion_name
    Champion_data["Win"] = win
    Champion_data["Lose"] = lose
    Champion_data["Win_ratio"] = Champion_data["Win"]/(Champion_data["Lose"]+Champion_data["Win"])
    Champion_data["Kill"] = kill
    Champion_data["Death"] = death
    Champion_data["Assist"] = assist
    Champion_data["KDA"] = (Champion_data["Kill"]+Champion_data["Assist"])/Champion_data["Death"]
    Champion_data["Gold"] = gold
    Champion_data["CS"] = cs
    Champion_data["Tower"] = tower
    Champion_data["Max_kill"] = max_kill
    Champion_data["Max_death"] = max_death
    Champion_data["Avg_attack_damage"] = avg_attack_damage
    Champion_data["Avg_defend_damage"] = avg_defend_damage
    Champion_data["double_Kill"] = double_kill
    Champion_data["triple_Kill"] = triple_kill
    Champion_data["quadra_Kill"] = quadra_kill
    Champion_data["Team"] = team
    Champion_data["User_name"] = user_name
    Champion_data["User_id"] = user_id
    Champion_data.head(10)
    return Champion_data

In [10]:
team = "KT"
user_name = "Mata"
user_id = "ktmata"

In [11]:
Champion_data = opgg_champ(team, user_name, user_id)

http://www.op.gg/summoner/champions/userName=ktmata


In [12]:
Champion_data

,Champion_Name,Win,Lose,Win_ratio,Kill,Death,Assist,KDA,Gold,CS,...,Max_kill,Max_death,Avg_attack_damage,Avg_defend_damage,double_Kill,triple_Kill,quadra_Kill,Team,User_name,User_id
0,Rakan,1.0,0.0,1.0,3.0,1.0,11.0,14.000000,7187.0,10.0,...,3.0,1.0,13859.0,6622.0,0.0,0.0,0.0,KT,Mata,ktmata
1,Thresh,1.0,0.0,1.0,1.0,4.0,17.0,4.500000,9748.0,51.0,...,1.0,4.0,28936.0,13936.0,0.0,0.0,0.0,KT,Mata,ktmata
2,Shen,1.0,0.0,1.0,4.0,6.0,34.0,6.333333,16787.0,260.0,...,4.0,6.0,176006.0,44090.0,0.0,0.0,0.0,KT,Mata,ktmata
3,Ezreal,1.0,0.0,1.0,8.0,5.0,16.0,4.800000,15197.0,196.0,...,8.0,5.0,181402.0,22474.0,0.0,0.0,0.0,KT,Mata,ktmata
4,Jarvan IV,1.0,0.0,1.0,4.0,3.0,11.0,5.000000,8945.0,17.0,...,4.0,3.0,122243.0,21451.0,0.0,0.0,0.0,KT,Mata,ktmata
5,Singed,1.0,0.0,1.0,3.0,10.0,15.0,1.800000,10575.0,174.0,...,3.0,10.0,113770.0,26657.0,0.0,0.0,0.0,KT,Mata,ktmata
6,Alistar,0.0,1.0,0.0,0.0,9.0,9.0,1.000000,6755.0,44.0,...,0.0,9.0,22475.0,24966.0,0.0,0.0,0.0,KT,Mata,ktmata


In [26]:
Total_List = []
for i in range(len(User_team)):
    Total_List.append([User_team[i],User_name[i],User_id[i]])
    

In [55]:
Total_List[:5]

[['ktrolster', 'score', '1110551del'],
 ['ktrolster', 'score', '좋은날'],
 ['ktrolster', 'score', '열심히할게요오'],
 ['ktrolster', 'mata', '시어니보고싶움'],
 ['ktrolster', 'smeb', 'smeb00']]

In [68]:
first = pd.DataFrame()

In [98]:
for i,total in enumerate(Total_List):
    team = total[0]
    user_name = total[1]
    user_id = total[2]
#     print(total[0])
#     print(total[1])
#     print(total[2])
    print(i)
    if i == 0:
        result = opgg_champ(team, user_name, user_id)
    elif i == 20:
        ;
    else:
        add_data = opgg_champ(team, user_name, user_id)
        result = result.append(add_data, ignore_index=True)
#     if i == 5:
#         break;
    time.sleep(2) # DDoS 대비

0
http://www.op.gg/summoner/champions/userName=1110551del
1
http://www.op.gg/summoner/champions/userName=좋은날
2
http://www.op.gg/summoner/champions/userName=열심히할게요오
3
http://www.op.gg/summoner/champions/userName=시어니보고싶움
4
http://www.op.gg/summoner/champions/userName=smeb00
5
http://www.op.gg/summoner/champions/userName=케이티pawn
6
http://www.op.gg/summoner/champions/userName=deft123456
7
http://www.op.gg/summoner/champions/userName=똥좀싸겟다
8
http://www.op.gg/summoner/champions/userName=ktdeft
9
http://www.op.gg/summoner/champions/userName=열심히좀해라
10
http://www.op.gg/summoner/champions/userName=ccppssnn
11
http://www.op.gg/summoner/champions/userName=kt
12
http://www.op.gg/summoner/champions/userName=춘봉박
13
http://www.op.gg/summoner/champions/userName=deft1
14
http://www.op.gg/summoner/champions/userName=matamatamata
15
http://www.op.gg/summoner/champions/userName=pawngod
16
http://www.op.gg/summoner/champions/userName=송하영팬
17
http://www.op.gg/summoner/champions/userName=totrace
18
http://www

146
http://www.op.gg/summoner/champions/userName=mvpian
147
http://www.op.gg/summoner/champions/userName=mvpmaha
148
http://www.op.gg/summoner/champions/userName=진주강씨29대손
149
http://www.op.gg/summoner/champions/userName=박학심문


In [99]:
result

,Champion_Name,Win,Lose,Win_ratio,Kill,Death,Assist,KDA,Gold,CS,...,Max_kill,Max_death,Avg_attack_damage,Avg_defend_damage,double_Kill,triple_Kill,quadra_Kill,Team,User_name,User_id
0,Kha'Zix,12.0,4.0,0.750000,9.4,5.1,7.2,3.254902,11354.0,26.5,...,18.0,14.0,114084.0,24140.0,0.0,0.0,0.0,ktrolster,score,좋은날
1,Nidalee,7.0,3.0,0.700000,7.2,4.6,8.2,3.347826,11495.0,30.8,...,11.0,10.0,135548.0,21943.0,0.0,0.0,0.0,ktrolster,score,좋은날
2,Rengar,9.0,0.0,1.000000,9.4,4.9,6.3,3.204082,12024.0,46.0,...,15.0,10.0,139832.0,23769.0,0.0,0.0,0.0,ktrolster,score,좋은날
3,Sejuani,6.0,3.0,0.666667,4.4,3.7,12.6,4.594595,12158.0,42.1,...,9.0,8.0,121522.0,34482.0,0.0,0.0,0.0,ktrolster,score,좋은날
4,Shyvana,5.0,4.0,0.555556,6.4,4.4,6.2,2.863636,10753.0,40.0,...,12.0,7.0,124124.0,24566.0,0.0,0.0,0.0,ktrolster,score,좋은날
5,Hecarim,3.0,4.0,0.428571,9.0,7.3,11.4,2.794521,12392.0,45.4,...,15.0,9.0,135886.0,33054.0,0.0,0.0,0.0,ktrolster,score,좋은날
6,Kindred,4.0,2.0,0.666667,6.7,4.5,8.3,3.333333,10998.0,34.7,...,10.0,11.0,130938.0,20222.0,0.0,0.0,0.0,ktrolster,score,좋은날
7,Ezreal,2.0,4.0,0.333333,7.5,5.7,8.2,2.754386,11792.0,99.3,...,12.0,8.0,143558.0,19588.0,0.0,0.0,0.0,ktrolster,score,좋은날
8,Elise,2.0,3.0,0.400000,6.8,5.2,6.4,2.538462,9934.0,27.6,...,12.0,6.0,95749.0,18689.0,0.0,0.0,0.0,ktrolster,score,좋은날
9,Rek'Sai,1.0,4.0,0.200000,5.2,3.8,5.2,2.736842,9497.0,30.2,...,8.0,5.0,109413.0,23233.0,0.0,0.0,0.0,ktrolster,score,좋은날


In [103]:
file_name = "test.csv"
result.to_csv(file_name)

In [56]:
Total_List[:5]

[['ktrolster', 'score', '1110551del'],
 ['ktrolster', 'score', '좋은날'],
 ['ktrolster', 'score', '열심히할게요오'],
 ['ktrolster', 'mata', '시어니보고싶움'],
 ['ktrolster', 'smeb', 'smeb00']]

In [92]:
for i,out in enumerate(result):
    if i == 0:
        first = result[0]
    else:
        first = first.append(result[i], ignore_index=True)
# data1 = result[0]
# data1.append(result[1],ignore_index=True)

In [93]:
first

,Champion_Name,Win,Lose,Win_ratio,Kill,Death,Assist,KDA,Gold,CS,...,Max_kill,Max_death,Avg_attack_damage,Avg_defend_damage,double_Kill,triple_Kill,quadra_Kill,Team,User_name,User_id
0,Kha'Zix,12.0,4.0,0.750000,9.4,5.1,7.2,3.254902,11354.0,26.5,...,18.0,14.0,114084.0,24140.0,0.0,0.0,0.0,ktrolster,score,좋은날
1,Nidalee,7.0,3.0,0.700000,7.2,4.6,8.2,3.347826,11495.0,30.8,...,11.0,10.0,135548.0,21943.0,0.0,0.0,0.0,ktrolster,score,좋은날
2,Rengar,9.0,0.0,1.000000,9.4,4.9,6.3,3.204082,12024.0,46.0,...,15.0,10.0,139832.0,23769.0,0.0,0.0,0.0,ktrolster,score,좋은날
3,Sejuani,6.0,3.0,0.666667,4.4,3.7,12.6,4.594595,12158.0,42.1,...,9.0,8.0,121522.0,34482.0,0.0,0.0,0.0,ktrolster,score,좋은날
4,Shyvana,5.0,4.0,0.555556,6.4,4.4,6.2,2.863636,10753.0,40.0,...,12.0,7.0,124124.0,24566.0,0.0,0.0,0.0,ktrolster,score,좋은날
5,Hecarim,3.0,4.0,0.428571,9.0,7.3,11.4,2.794521,12392.0,45.4,...,15.0,9.0,135886.0,33054.0,0.0,0.0,0.0,ktrolster,score,좋은날
6,Kindred,4.0,2.0,0.666667,6.7,4.5,8.3,3.333333,10998.0,34.7,...,10.0,11.0,130938.0,20222.0,0.0,0.0,0.0,ktrolster,score,좋은날
7,Ezreal,2.0,4.0,0.333333,7.5,5.7,8.2,2.754386,11792.0,99.3,...,12.0,8.0,143558.0,19588.0,0.0,0.0,0.0,ktrolster,score,좋은날
8,Elise,2.0,3.0,0.400000,6.8,5.2,6.4,2.538462,9934.0,27.6,...,12.0,6.0,95749.0,18689.0,0.0,0.0,0.0,ktrolster,score,좋은날
9,Rek'Sai,1.0,4.0,0.200000,5.2,3.8,5.2,2.736842,9497.0,30.2,...,8.0,5.0,109413.0,23233.0,0.0,0.0,0.0,ktrolster,score,좋은날
